# Choosing dispersion correction

Since the ACE parametrization is fitted to PBE data, it does not consider long-range dispersion interactions that are important for many carbon structures. This notebook describes how it is possible to inlcude D2, D3 or D4 corrections by combining a lammps calculator with the desired dispersion correction.

To use a lammps calculator, lammps is compiled with shared libraries, lammps python module: https://docs.lammps.org/Python_install.html and ML-PACE package: https://docs.lammps.org/Packages_details.html#pkg-ml-pace

This can be done by following the steps:

```
mkdir build_lib
cd build_lib
cmake -D BUILD_SHARED_LIBS=ON -D BUILD_MPI=ON -D PKG_MPIIO=ON -D LAMMPS_EXCEPTIONS=yes -D PKG_MANYBODY=ON -D PKG_MISC=ON -D PKG_MISC=ON -D PKG_EXTRA-COMPUTE=ON -D PKG_EXTRA-DUMP=ON -D PKG_EXTRA-FIX=ON -D PKG_EXTRA-PAIR=ON -D PKG_ML-PACE=ON ../cmake
make
cp liblammps${SHLIB_EXT}* ../src 
cd ../src
make install-python 
mkdir -p $CONDA_PREFIX/include/lammps
cp library.h $CONDA_PREFIX/include/lammps
cp liblammps${SHLIB_EXT}* $CONDA_PREFIX/lib/
cd ..
```

If everyone is compiled properly, the python API of lammps can be tested:
```
from lammps import lammps
lmp = lammps()
```

which should run witohut errors.

Now, `ase.calculators.lammpslib/LAMMPSlib` can be used as an ASE calculator


In [1]:
from ase.calculators.lammpslib import LAMMPSlib
from ase import Atoms
import numpy as np

Define a graphite structure to make test calculations

In [2]:
def get_graphite(a0,c=6.7):
    return Atoms("C4", scaled_positions=[(0.00,0.00,0.00),(0.00,0.00,0.5),(0.33,0.67,0.0),(0.67,0.33,0.5)], 
           cell=[[a0, 0.0, 0.0], [-0.5 * a0, a0 * np.sqrt(3)/2, 0.0], [0.0, 0.0, c]],pbc=True)
a0 = 2.4768
graphite = get_graphite(a0)

## 1. Base ACE calculator 

The base ACE calculator ($ACE_{PBE}$) can be defined simply in lammps as following:

In [3]:
model = "potential_file/c_ace.yace"

cmds = [
    "pair_style pace",
    "pair_coeff * * %s C"%model
]

calc_pace = LAMMPSlib(lmpcmds=cmds)

In [4]:
%%time

graphite.calc = calc_pace

pace_energy = graphite.get_potential_energy()/len(graphite)
pace_energy

print("The pace energy is %.2f eV/atom"%(pace_energy))


The pace energy is -9.04 eV/atom
CPU times: user 141 ms, sys: 23.4 ms, total: 164 ms
Wall time: 189 ms


## D2

The D2 correction is provided in the `potential_file` directory as a lammps `pair_style table` format. It can be readily merged with $ACE_{PBE}$ using `hybrid/overlay` pair_style of lammps

In [5]:
model = "potential_file/c_ace.yace"
d2_table = "potential_file/d2.table"


cmds = [
    "pair_style hybrid/overlay pace table linear 20000",
    "pair_coeff * * pace %s C"%model,"pair_coeff * * table %s D2 9.0"%d2_table
]

calc_d2 = LAMMPSlib(lmpcmds=cmds)

In [6]:
%%time

graphite.calc = calc_d2

d2_energy = graphite.get_potential_energy()/len(graphite)

print("The total pace+d2 energy is %.2f eV/atom"%(d2_energy))

The total pace+d2 energy is -9.16 eV/atom
CPU times: user 183 ms, sys: 48.2 ms, total: 231 ms
Wall time: 166 ms


## D3

The D3 implementation can be downloaded and installed directly from Grimme et al. https://dftd3.readthedocs.io/en/latest/

In [7]:
from dftd3.ase import DFTD3

d3 = DFTD3(method="PBE", damping="d3zero")

In [8]:
%%time

graphite.calc = d3
d3_energy = graphite.get_potential_energy()/len(graphite)

print("The total energy is %.2f eV/atom where D3 contribution is %.2f eV/atom"%(pace_energy + d3_energy, d3_energy))


The total energy is -9.13 eV/atom where D3 contribution is -0.09 eV/atom
CPU times: user 4.08 s, sys: 114 ms, total: 4.19 s
Wall time: 4 s


## D4

The D4 implementation can be downloaded and installed directly from Grimme et al. https://dftd4.readthedocs.io/en/latest/reference/ase.html


In [9]:
from dftd4.ase import DFTD4

d4 = DFTD3(method="PBE", damping="d3zero")

In [10]:
%%time

graphite.calc = d4
d4_energy = graphite.get_potential_energy()/len(graphite)

print("The total energy is %.2f eV/atom where D4 contribution is %.2f eV/atom"%(pace_energy + d4_energy, d4_energy))


The total energy is -9.13 eV/atom where D4 contribution is -0.09 eV/atom
CPU times: user 4 s, sys: 8.48 ms, total: 4.01 s
Wall time: 4.03 s
